In [1]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [4]:
#importing necessary libraries
import openpyxl
import cv2 as cv
import time
import geocoder
import os

In [5]:
#reading label name from obj.names file
class_name = []
with open(os.path.join("project_files",'obj.names'), 'r') as f:
    class_name = [cname.strip() for cname in f.readlines()]

In [6]:
#importing model weights and config file
#defining the model parameters
net1 = cv.dnn.readNet('project_files/yolov4_tiny.weights', 'project_files/yolov4_tiny.cfg')
net1.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
net1.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA_FP16)
model1 = cv.dnn_DetectionModel(net1)
model1.setInputParams(size=(640, 480), scale=1/255, swapRB=True)

In [7]:
#defining the video source (0 for camera or file name for video)
cap = cv.VideoCapture("test.mp4") 

In [8]:
#defining the video source (0 for camera or file name for video)
#cap = cv.VideoCapture(0) 
width  = cap.get(3)
height = cap.get(4)
result = cv.VideoWriter('result.avi', 
                         cv.VideoWriter_fourcc(*'MJPG'),
                         10,(int(width),int(height)))


In [9]:
#defining parameters for result saving and get coordinates
#defining initial values for some parameters in the script
g = geocoder.ip('me')
result_path = "pothole_coordinates"
starting_time = time.time()
Conf_threshold = 0.5
NMS_threshold = 0.4
frame_counter = 0
i = 0
b = 0

In [10]:
pothole_scores=[]
file_list=[]
location=[]
#detection loop
while True:
    ret, frame = cap.read()
    frame_counter += 1
    if ret == False:
        break
    #analysis the stream with detection model
    classes, scores, boxes = model1.detect(frame, Conf_threshold, NMS_threshold)
    for (classid, score, box) in zip(classes, scores, boxes):
        label = "pothole"
        x, y, w, h = box
        recarea = w*h
        area = width*height
        #drawing detection boxes on frame for detected potholes and saving coordinates txt and photo
        if(len(scores)!=0 and scores[0]>=0.7):
            if((recarea/area)<=0.1 and box[1]<600):
                cv.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 1)
                cv.putText(frame, "%" + str(round(scores[0]*100,2)) + " " + label, (box[0], box[1]-10),cv.FONT_HERSHEY_COMPLEX, 0.5, (255,0,0), 1)
                if(i==0):
                    pothole_scores.append(scores[0])
                    cv.imwrite(os.path.join(result_path,'pothole'+str(i)+'.jpg'), frame)
                    file_list.append('pothole'+str(i)+'.jpg')
                    with open(os.path.join(result_path,'pothole'+str(i)+'.txt'), 'w') as f:
                        f.write(str(g.latlng))
                        location.append(g.latlng)
                        i=i+1
                if(i!=0):
                    if((time.time()-b)>=2):
                        pothole_scores.append(scores[0])
                        cv.imwrite(os.path.join(result_path,'pothole'+str(i)+'.jpg'), frame)
                        file_list.append('pothole'+str(i)+'.jpg')
                        with open(os.path.join(result_path,'pothole'+str(i)+'.txt'), 'w') as f:
                            f.write(str(g.latlng))
                            location.append(g.latlng)
                            b = time.time()
                            i = i+1
    #writing fps on frame
    endingTime = time.time() - starting_time
    fps = frame_counter/endingTime
    cv.putText(frame, f'FPS: {fps}', (20, 50),
               cv.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
    #showing and saving result
    cv.imshow('frame', frame)
    result.write(frame)
    key = cv.waitKey(1)
    if key == ord('q'):
        break
#end
cv.destroyAllWindows()
cap.release()
result.release()

In [11]:
print(*file_list, sep = ", ") 

pothole0.jpg, pothole1.jpg, pothole2.jpg, pothole3.jpg, pothole4.jpg, pothole5.jpg, pothole6.jpg, pothole7.jpg, pothole8.jpg, pothole9.jpg, pothole10.jpg, pothole11.jpg, pothole12.jpg


In [12]:
print(*pothole_scores, sep = ", ") 

0.82108444, 0.82108444, 0.8481602, 0.8277846, 0.7001757, 0.90311116, 0.94473284, 0.9340939, 0.86474365, 0.7750769, 0.7363681, 0.7597128, 0.87666


In [13]:
print(*location, sep = ", ") 

[13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785], [13.0878, 80.2785]


In [14]:
file_list_len = len(file_list)
print(file_list_len)

13


In [15]:
pothole_num = len(pothole_scores)
print(pothole_num)

13


In [16]:
wBook = openpyxl.load_workbook('mydata.xlsx')
sheet = wBook.active
for i in range(0,len(file_list)):
    data = [file_list[i], pothole_scores[i], location[0][0],location[0][1]]
    sheet.append(data)
    wBook.save('mydata.xlsx')
print('All records inserted successfully !')

All records inserted successfully !


In [22]:
fromaddr = "arvinthus2410@gmail.com"
toaddr = "arvinth33gobi@gmail.com"

# instance of MIMEMultipart
msg = MIMEMultipart()

# storing the senders email address
msg['From'] = fromaddr

# storing the receivers email address
msg['To'] = toaddr

# storing the subject
msg['Subject'] = "Mail - pothole detected reg."

body="Body of the mail"

# attach the body with the msg instance
msg.attach(MIMEText(body, 'plain'))

# open the file to be sent
filename = "pothole_report.xlsx"
attachment = open("C:/Users/arvin/Downloads/pothole/pathhole/mydata.xlsx", 'rb')

#     # instance of MIMEBase and named as p
p = MIMEBase('application', 'octet-stream')

#     # To change the payload into encoded form
p.set_payload((attachment).read())

#     # encode into base64
encoders.encode_base64(p)

p.add_header('Content-Disposition', "attachment; filename= %s" % filename)

#     # attach the instance 'p' to instance 'msg'
msg.attach(p)

dir_path = "C:/Users/arvin/Downloads/pothole/pathhole/pothole_coordinates"
for f in file_list:  # add files to the message
        file_path = os.path.join(dir_path, f)
        attachment = MIMEApplication(open(file_path, "rb").read(), _subtype="txt")
        attachment.add_header('Content-Disposition','attachment', filename=f)
        msg.attach(attachment)


# creates SMTP session
s = smtplib.SMTP('smtp.gmail.com', 587)

# start TLS for security
s.starttls()

# Authentication
s.login(fromaddr, "hlgh lgqa hrwg biwx")

# Converts the Multipart msg into a string
text = msg.as_string()

# sending the mail
s.sendmail(fromaddr, toaddr, text)

# terminating the session
s.quit()


(221,
 b'2.0.0 closing connection p5-20020a170902bd0500b001b8af7f632asm4010150pls.176 - gsmtp')